In [17]:

%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [18]:
%hive_start

In [19]:

%timeout 300

In [20]:
!hdfs dfs -ls /tmp

Found 3 items
drwxrwx---   - root supergroup          0 2020-02-11 01:46 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 01:54 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 02:22 /tmp/hivep7


In [21]:
!hdfs dfs -mkdir /tmp/hivep7

mkdir: `/tmp/hivep7': File exists


In [22]:
!hdfs dfs -ls /tmp/

Found 3 items
drwxrwx---   - root supergroup          0 2020-02-11 01:46 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 01:54 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 02:22 /tmp/hivep7


In [23]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/hivep7/

In [24]:
!hdfs dfs -copyFromLocal tbl1.csv /tmp/hivep7/

In [25]:
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/hivep7

Found 3 items
-rwxr-xr-x   1 root supergroup         29 2020-02-11 02:22 /tmp/hivep7/000000_0
-rw-r--r--   1 root supergroup        397 2020-02-11 02:27 /tmp/hivep7/tbl0.csv
-rw-r--r--   1 root supergroup        282 2020-02-11 02:27 /tmp/hivep7/tbl1.csv


In [26]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 21.648 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.334 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.506 seconds


In [27]:
%%hive
DROP TABLE IF EXISTS pt7;
CREATE TABLE pt7
AS
SELECT c2, collect_list((c1)) AS lista
FROM tbl0
GROUP BY c2;

INSERT OVERWRITE DIRECTORY '/tmp/hivep7'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT * FROM pt7;

DROP TABLE IF EXISTS pt7;
OK
Time taken: 0.079 seconds
CREATE TABLE pt7
AS
SELECT c2, collect_list((c1)) AS lista
FROM tbl0
GROUP BY c2;
Query ID = root_20200211022832_527c0e8d-bb28-46e3-b446-ae89e24cbfba
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1581385522257_0003, Tracking URL = http://80b7b573c52f:8088/proxy/application_1581385522257_0003/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1581385522257_0003
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-11 02:28:56,693 Stage-1 map = 0%,  reduce = 0%
2020-02-11 02:29:04,996 Stage-1 map = 100%,  reduce = 0%, Cumulativ

In [28]:
!hdfs dfs -cat /tmp/hivep7/000000_0 

A,6
B,7:9:10
C,2:5:8
D,1:3:4


In [30]:
!hdfs dfs -copyToLocal /tmp/hivep7/000000_0 output;

In [31]:
%%hive
DROP TABLE pt7;
%hive_quit

DROP TABLE pt7;
OK
Time taken: 0.41 seconds
%hive_quit
